In [2]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gradient_descent_wandb import *
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np
import wandb
from types import SimpleNamespace
import random

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

X = train_images.reshape(train_images.shape[0], -1).T/225
Y = train_labels
X_test = test_images.reshape(test_images.shape[0], -1).T/225
Y_test = test_labels

validation_ratio = 0.1 #percentage of data for validation
num_validation_samples = int(validation_ratio * X.shape[1])
indices = np.random.permutation(X.shape[1]) #shuffling the indices

validation_indices = indices[:num_validation_samples]
training_indices = indices[num_validation_samples:]

X_train = X[:, training_indices]
Y_train = Y[training_indices]
X_val = X[:, validation_indices]
Y_val = Y[validation_indices]

print("Number of training samples: " + str(X_train.shape[1]))
print("Number of validation samples: " + str(X_val.shape[1]))
print("Number of testing samples: " + str(X_test.shape[1]))

wandb.login(key='4734e60951ce310dbe17484eeeb5b3366b54850f')

sweep_config = {
    'method': 'bayes',
    'name': 'sweep mse jupiter corrected lrs',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'hidden_layers':{
            'values':[3,4,5]
        },
        'activation': {
            'values': ['sigmoid','relu','tanh']
        },
        'loss': {
            'values': ['mse']
        },
        'weight_decay': {
            'values': [0,0.0005,0.5]
        },
        'optimizer': {
            'values': ['sgd','momentum','nesterov','rmsprop','adam','nadam']
        },
        'lr': {
            'values': [1e-3,1e-4]
        },
        'batch_size': {
            'values': [16,32,64] 
        },
        'w_init': {
            'values':['glorot','random'] 
        },
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='CS6910_assignment_1')
wandb.init(project='CS6910_assignment_1', entity='sumanta_roy')

def main():
    '''
    WandB calls main function each time with different combination.

    We can retrive the same and use the same values for our hypermeters.

    '''

    with wandb.init() as run:

        run_name="-ac_"+wandb.config.activation+" -hs_"+str(wandb.config.hidden_size)+" -hl_"+str(wandb.config.hidden_layers)+" -epch_"+str(wandb.config.epochs)+" -bs_"+str(wandb.config.batch_size)+" -init_"+str(wandb.config.w_init)+" -lr_"+str(wandb.config.lr)+" -l2d_"+str(wandb.config.weight_decay)
        wandb.run.name=run_name
        #obj=NN(wandb.config['num_layers'],wandb.config['hidden_size'])

        max_epochs=wandb.config.epochs
        no_hidden_layers=wandb.config.hidden_layers
        size_of_hidden_layer=wandb.config.hidden_size
        weight_decay = wandb.config.weight_decay
        alpha=wandb.config.lr
        opt=wandb.config.optimizer #'sgd','momentum','nesterov','rmsprop','adam','nadam'
        batch_size=wandb.config.batch_size
        weight=wandb.config.w_init #glorot,random
        activation=wandb.config.activation #sigmoid,tanh,relu

        input_size,output_size = 784,10
        layer_sizes = [input_size] + [size_of_hidden_layer] * no_hidden_layers + [output_size]
        beta=0.9
        epsilon=1e-4
        beta1=0.9
        beta2=0.999

        w = 50
        
        if opt == 'sgd':
          params = stochastic_gradient_descent(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val, max_epochs, alpha, layer_sizes, batch_size)
        elif opt == 'momentum':
          params = momentum_gradient_descent(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val, max_epochs, alpha, beta, layer_sizes, batch_size)
        elif opt == 'nesterov':
          params = nesterov_gradient_descent(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val, max_epochs, alpha, beta, layer_sizes, batch_size)
        elif opt == 'rmsprop':
          params = stochastic_gradient_descent_with_RMSProp(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val,max_epochs, alpha, layer_sizes, batch_size, epsilon, beta)
        elif opt == 'adam':
          params = stochastic_gradient_descent_with_adam(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val, max_epochs, alpha, layer_sizes, batch_size, epsilon, beta1, beta2)
        elif opt == 'nadam':
          params = stochastic_gradient_descent_with_nadam(w,weight, activation, weight_decay, X_train, Y_train, X_val, Y_val, max_epochs, alpha, layer_sizes, batch_size, epsilon, beta1, beta2)
        else:
          raise ValueError(f"Invalid optimizer option: {opt}")

wandb.agent(sweep_id, function=main,count=100) # calls main function for count number of times.
wandb.finish()


Number of training samples: 54000
Number of validation samples: 6000
Number of testing samples: 10000


wandb: Currently logged in as: sumanta_roy. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/sumanta/.netrc


Create sweep with ID: etqoz0cb
Sweep URL: https://wandb.ai/sumanta_roy/CS6910_assignment_1/sweeps/etqoz0cb


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 7g6bnocw with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 128
wandb: 	loss: mse
wandb: 	lr: 0.0001
wandb: 	optimizer: sgd
wandb: 	w_init: random
wandb: 	weight_decay: 0.5


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


...

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_accuracy,▁▃▂▁▃▁▂█▃▆▇▄▆▅▃▄▅▄▇▁▆▃▆▆▇▆▅█▅▅▆▃▅▇▆█▆▇▅▇
train_loss,█▆▆▆▅▅▄▂▄▃▃▄▃▃▄▅▃▂▃▅▂▄▃▃▃▂▃▁▄▃▃▅▃▂▂▁▂▂▂▂
val_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████████
validation_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,10
train_accuracy,93.75
train_loss,0.01508
val_accuracy,84.36667
validation_loss,0.0226


wandb: Agent Starting Run: 0bycu17n with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 64
wandb: 	loss: mse
wandb: 	lr: 0.001
wandb: 	optimizer: nadam
wandb: 	w_init: random
wandb: 	weight_decay: 0.0005


Epoch: 1/10; Train Loss: 0.07485152304824676; Val Loss: 0.036126857713337866
Epoch: 2/10; Train Loss: 0.06773496844994244; Val Loss: 0.03233072781462405


wandb: Ctrl + C detected. Stopping sweep.


Epoch: 3/10; Train Loss: 0.06296246308790719; Val Loss: 0.030074715569054115
